In [1]:
import pdfplumber
import json
import pandas as pd
from copy import deepcopy

# File path for store data
store_data_path = '~/Desktop/'
store_data_file = 'StoreReports-1.pdf'
store_data = store_data_path + store_data_file

# File path for weather data
weather_data_path = '~/Documents/GitHub/Order-Assistant/WeatherData.csv'

PAGES = 5
data_columns = ['Begin', 'Delivery', 'End', 'Used', 'Cost', '% of sales']


In [2]:
class items:
    identified = False
    begin = 0.00
    delivery = 0.00
    end = 0.00
    used = 0.00
    cost = 0.00
    net = 0.00

    def __init__(self, name) -> str:
        self.name = name

    def checked(self, id) -> bool:
        self.identified = id

    def addData(self, begin, delivery, end, used, cost, net):
        self.begin = begin
        self.delivery = delivery
        self.end = end
        self.used = used
        self.cost = cost
        self.net = net


In [3]:
# list of all items 
drinks = {
    '591ml Pepsi' : False,
    '591ml Diet Pepsi' : False,
    '591ml Mountain Dew' : False,
    '591ml Diet Mtn Dew' : False,
    '591ml Aquafina' : False,
    '591ml Dr Pepper' : False,
    '591ml Mug Root Beer' : False,
    '591ml Schw Gngr Ale' : False,
    '591ml WCherry Pepsi' : False,
    '591ml Crush Orange' : False,
    '591ml Seven Up' : False,
    '591ml Brsk IceT' : False,
    '2Ltr Pepsi' : False,
    '2Ltr Diet Pepsi' : False,
    '2Ltr Mountain Dew' : False,
    '2Ltr Dt Mtn Dew' : False,
    '2Ltr Dr Pepper' : False,
    '2Ltr Mug Root Beer' : False,
    '2Ltr Schw Gngr Ale' : False,
    '2Ltr Crush Orange' : False,
    '2Ltr Brsk SwTea Lm' : False,
    '2Ltr Seven Up' : False,
    '355ml Pepsi' : False,
    '355ml Diet Pepsi' : False,
    '355ml Diet Mtn Dew' : False,
    '355ml Dr Pepper' : False,
    '355ml Schw Gngr Ale' : False,
    '355ml Brsk SwTea Lm' : False,
    '355ml Seven Up' : False,
    '12oz Welch Gr Soda' : False
}

cheese = {
    'Cheddar Cheese' : False,
    'Three Cheese Blend' : False,
    'String Cheese' : False,
    'Agropur Cheese' : False,
    '2 Cheese' : False,
    '3 Cheese' : False,
    'Feta Cheese' : False,
    'Philly Whipped Cream Chee' : False

}

dough = {
    'Dough Tray 10': False,
    'Dough Tray 12': False,
    'Dough Tray 14': False,
    'Dough Tray 16': False,
    'Chocolate Chip Cookie': False,
    'Choc Chip Brownie': False,
    'Cinnamon Pull Apart': False,
    'Thin Crust 14': False,
    'Gluten Free Crust': False
}

# There are two pepperonis in the log, needs work
meats = {
    'Pepperoni' : False,
    'Bacon' : False,
    'Salami' : False,
    'Italian Sausage' : False,
    'Anchovies' : False,
    'Grilled Chicken' : False,
    'Chicken Popper' : False,
    'Salami' : False,
    'Canadian Bacon' : False,
    'Philly Cheesesteak' : False,
    'Sliced Spicy Meatballs' : False,
    'Meatball' : False,
    'Beef' : False,
    'Sausage' : False,
    'Italian Sausage' : False,
    'Roasted Wings' : False,
    'Donair Topping' : False
}

produce = {
    'Banana Peppers' : False,
    'Black Olives' : False,
    'Pineapple Lg Can' : False,
    'Mushrooms' : False,
    'Fresh Spinach' : False,
    'Green Peppers' : False,
    'Onions' : False,
    'Tomatoes' : False,
    'Banana Peppers' : False,
    'Pepperoncini Peppers' : False,
    'Jalapeno Peppers' : False,
    'Green Olives' : False
}

sauces = {
    'Bulk Ranch Sauce': False,
    'Pizza Sauce': False,
    'Jug Garlic Sauce': False,
    'Garlic Sauce Cups': False,
    'Pizza Sauce Cups': False,
    'Cheese Sauce Cups': False,
    'Ranch Sauce Cups': False,
    'Barbecue Sauce Cups': False,
    'Buffalo Sauce Cups': False,
    'Honey Must Sauce Cups': False,
    'Blue Cheese Sauce Cups': False,
    'Barbecue Sauce': False,
    'Garlic Parmesan Sauce Jug': False,
    'Buffalo Sauce': False,
    'Pouch Honey Chptl': False,
    'BBQ Bulk': False,
    'Alfredo Sauce': False,
    'Spinach Alfredo Sauce': False,
    'Cream Cheese Flavored Ici': False,
    'Creamy Garlic Dipping Cup': False,
    'Donair Sauce': False,
    'Hellman\'s Bulk Ranch': False
}

seasoning = {
    'Italian Seasoning' : False,
    'Seasoning Packets' : False,
    'Parmesan' : False,
    'Crushed Red Pepper' : False,
    'Dustinator' : False
}

boxes = {
    'Pizza Box 10': False,
    'Pizza Box 12': False,
    'Pizza Box 14': False,
    'Pizza Box 16': False,
    'Pizza Slice Box': False
}

In [4]:
category_dict = {
    'BEVERAGES' : drinks,
    'CHEESE' : cheese,
    'MEATS' : meats,
    'PRODUCE' : produce,
    'DOUGH' : dough,
    'SAUCES' : sauces,
    'SEASONS/FLOUR' : seasoning,
    'BOXES' : boxes
}

In [5]:
# creates a josn with all the data : category { items { data { 'Begin' : 0.00, ... }}}
# TODO Fix : only uses data from the last row
def logData(inventory, data_in):
    for category in category_dict:
        items = {}
        for item in category_dict[category]:
            data = {}
            for i in range(len(data_columns)):
                try :
                    x = deepcopy(float(data_in[i]))
                except:
                    # print(f'* ERROR * failed to convert {data_in[i]} data for {item} to float')
                    x = 0.00
                
                data[data_columns[i]] = x
                
            items[item] = deepcopy(data)
            
        inventory[category] = items
        

In [6]:
def extrator(path: str) -> str:
    date = 'date'
    category = ''
    detail_flag = False # lets program know when to start recording values
    detail_counter = 0
    details = [0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    word_count = 0
    inventory = {}

    for page_no in range(PAGES):
        with pdfplumber.open(path) as pdf:
            page = pdf.pages[page_no]
            text = page.extract_text()

        word = ''
        space_counter = 0

        for character in text:
            if ord(character) != 160:                                               # 160 is the hex value for ' '
                word += character
                space_counter = 0

            elif len(word) > 0: 
                word += ' '
                space_counter += 1
                if space_counter == 2:                                              # word ends if there are more than one space between words
                    word = word.strip()
                    word_count += 1
                    # print(f'{word} {word_count}')
                    
                    if word in category_dict:                                       # a new category will only appear once all items have been read
                        for item in category:
                            if not category[item]:
                                print(f'** WARNING ** did not log {item}')
                                return ValueError

                        category = category_dict[word]

                    elif word in category:
                        detail_flag = True
                        if detail_counter == 6:                                     # Log all the items when inventory information is collected
                            logData(inventory, details)

                        detail_counter = 0
                        category[word] = True

                    elif detail_counter > 5:
                        detail_flag = False

                    elif detail_flag and detail_counter < 6:
                        details[detail_counter] = word
                        detail_counter += 1

                    # characer 341
                    elif page_no == 0 and word_count == 6:
                        try:
                            word = word.split(' \xad ')[0]
                            m,d,y = word.split('/')
                            date = ('-').join([y,m,d])
                        except:
                            pass

                    word = ''    
                    space_counter = 0       

    # print('File Complete')
    return(date, inventory)

In [7]:
def getWeatherDataAsJSON(path):
    data = pd.read_csv(path)

    full_info = {}
    day = data['0']
    temp = data['1']

    for i in range(100):
        full_info[day[i]] = temp[i]

    return full_info


In [8]:
data = dict()
def mergeData(weather_data : dict, store_data : dict, date : str):
    
    try: 
        store_data['AvgTempC'] = weather_data[date]
        print(weather_data[date])
    except KeyError: 
        print(f'Date {date} has no weather data')
        store_data['AvgTempC'] = None

    import pdb; pdb.set_trace() 
    data[date] = store_data

    # date : cate: ...


In [9]:
# extract data from all data files and save it into one large JSON file
weather_data = getWeatherDataAsJSON(weather_data_path)

NUMBER_OF_FILES = 3

for i in range (NUMBER_OF_FILES):
    file_no = i + 1
    store_data_file = f'StoreReports-{file_no}.pdf'
    try :
        # date, week_data = extrator(store_data_path + store_data_file)
        date, week_data = extrator(store_data_file)

    except :
        print(f'failed to read file StoreReports-{file_no}.pdf')
        continue
    
    print(date)
    data = mergeData(weather_data, week_data, date)


2021-09-13
Date 2021-09-13 has no weather data
> <ipython-input-8-e69bef01ed0c>(12)mergeData()
     10 
     11     import pdb; pdb.set_trace()
---> 12     data[date] = store_data
     13 
     14     # date : cate: ...

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
2021-09-06
Date 2021-09-06 has no weather data
> <ipython-input-8-e69bef01ed0c>(12)mergeData()
     10 
     11     import pdb; pdb.set_trace()
---> 12     data[date] = store_data
     13 
     14     # date : cate: ...



In [ ]:
print(week_data['BEVERAGES']['591ml Diet Pepsi'])

{'Begin': 183.0, 'Delivery': 200.0, 'End': 171.0, 'Used': 212.0, 'Cost': 131.44, '% of sales': 0.65}


In [ ]:
# TODO everything is saved with the values for Pizza box 16
# save data to file
def saveToFile(data, num):
    with open(f'data/data{num}.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)